In [24]:
import napari
import napari_stress
import napari_process_points_and_surfaces as nppas
import napari_segment_blobs_and_things_with_membranes as nsbatwm
import napari_stl_exporter as npstl
import vedo
import copy
from skimage import data, color, filters, transform
from skimage.transform import rescale, resize, downscale_local_mean
import os
import skimage.io
import pandas as pd
import os
import numpy as np
from napari.utils import nbscreenshot

from napari_stress import reconstruction
import matplotlib.pyplot as plt

In [15]:
# create viewer
viewer = napari.Viewer(ndisplay=3)

In [25]:
num_droplet = 9


DATA_ROOT = os.path.join("C:\\","Users","Heloise","Documents","Data","Naked1","%s" %num_droplet)

image_ini = skimage.io.imread(os.path.join(DATA_ROOT, 'droplet%s.tif' %num_droplet)) #Raw image
Dec = skimage.io.imread(os.path.join(DATA_ROOT, 'Dec10_median.tif' ))   #Deconvoluted + median filtered image
Edge = skimage.io.imread(os.path.join(DATA_ROOT, 'edges.tif' ))  #After 3D sobel edges filter is applied

In [26]:
# Config

#Voxel size in µm (with Pixel_size)
%run Pixel_Size.ipynb

voxelsize = read_tiff_voxel_size(os.path.join(DATA_ROOT, 'Dec10_median.tif'))
voxelsize = np.asarray(voxelsize)
target_voxelsize = voxelsize[1]

print(voxelsize, target_voxelsize)

'''
vsz = 0.1579
vsy = 0.1245
vsx = 0.1245
target_voxelsize = 0.1245  #microns #microns

voxelsize = np.asarray([vsz, vsx, vsy])

'''
scaling_factors = voxelsize / target_voxelsize

print(scaling_factors)

[0.19979493 0.12449412 0.12449412] 0.12449411815089384
[1.60485439 1.         1.        ]


In [ ]:
n_frames = Dec.shape[0]

viewer.add_image(image_ini, name='Raw', scale = voxelsize)
viewer.add_image(Dec, name='deconvoluted_data', scale = voxelsize)
viewer.add_image(Edge, name='edge', scale = voxelsize)


In [38]:
#Parameters for refinement

iterations_PSF = 10
smoothing_sigma = 1
n_smoothing_iterations = 10
resampling_length = 2.5
n_tracing_iterations = 10
trace_length = 12
outlier_tolerance = 0.5
Blurred = True


In [39]:
#Rescale

scale = np.asarray([target_voxelsize] * 3)  
rescaled_image = transform.rescale(Edge[0], scaling_factors,anti_aliasing=False)

resampled_image_layer = viewer.add_image(rescaled_image, scale = scale)



In [ ]:
#It is sometimes necessary to blurr the image if it's still too noisy
blurred_image = filters.gaussian(rescaled_image, sigma=smoothing_sigma)
viewer.add_image(blurred_image, scale = scale) 

In [40]:
#Binarization and marching cube

threshold = filters.threshold_otsu(blurred_image)
binarized_image = blurred_image > threshold*0.9
label_image = nsbatwm.connected_component_labeling(binarized_image)
binarized_image_layer = viewer.add_labels(label_image, scale = scale)
surface = nppas.largest_label_to_surface(label_image)

viewer.add_surface(surface, scale = scale)

# Smooth surface to simplify the guess of the first points
mesh_vedo = vedo.mesh.Mesh((surface[0], surface[1])).clean()
n_smoothing_iterations = 10
surface_smoothed = nppas.smooth_surface((mesh_vedo.points(), mesh_vedo.faces()),number_of_iterations=n_smoothing_iterations)
smoothed_image_layer = viewer.add_surface(surface_smoothed, scale = scale)


points_first_guess = nppas.sample_points_from_surface(surface_smoothed, distance_fraction=0.02)  #adapt the distance fraction to have 2000-3000 points
viewer.add_points(points_first_guess, size = 1, name = "points first guess", scale = scale)

points = copy.deepcopy(points_first_guess)

<Surface layer 'surface' at 0x2234d8b7670>

In [43]:
from napari_stress._reconstruction.toolbox import _resample_pointcloud
 
resampled_points = _resample_pointcloud(points, sampling_length=resampling_length)

print(len(resampled_points))
viewer.add_points(resampled_points, size = 1, face_color = 'cyan', name = "resampled_points", scale = scale)
#print(resampled_points)


2260


<Points layer 'resampled_points' at 0x223579456a0>

In [44]:
sampling_distance = 0.02  # microns
fit_type = 'fancy'   
edge_type = 'surface'

#surface_density = 0.35  # points/micron^2
#curvature_radius = 10  # microns

In [31]:
from napari_stress._reconstruction.toolbox import _resample_pointcloud

for i in range(n_tracing_iterations):
    resampled_points = _resample_pointcloud(points, sampling_length=resampling_length)
    #viewer.add_points(resampled_points, size = 2, name = "res")
    traced_points, traced_vectors = reconstruction.trace_refinement_of_surface(
        blurred_image,
        resampled_points,
        selected_fit_type=fit_type,
        selected_edge=edge_type,
        trace_length=trace_length,
        sampling_distance=sampling_distance,
        remove_outliers=True,
        outlier_tolerance=outlier_tolerance,
        interpolation_method="linear")
        points = traced_points[0]
        #print(len(points))
    


1979
1863
1825
1795
1787
1775
1764
1761
1763
1759


In [48]:
viewer.add_points(traced_points[0], size = 1.5, edge_color = "blue", name = "refined_points", scale = scale)
viewer.add_vectors(traced_vectors[0], **traced_vectors[1], edge_color='red', scale=scale)


refined_points


In [11]:
#Save the data in an excel file


file_name = '%s_reconstruction.xlsx' %(num_droplet)
saving_path = DATA_ROOT.replace("\\", "/")
excel_file = '%s/%s' %(saving_path, file_name)

name_info_sup = ['iterations_PSF','smoothing_sigma', 'nb_smoothing', 'nb_points_first_guess', 'resampling_length', 'vector_length', 'Blurred', 'n_iterations', 'outlier_tolerance', 'interpolation_method', 'target_voxel_size']
info_sup = [iterations_PSF, smoothing_sigma, n_smoothing_iterations, len(points_first_guess), resampling_length, trace_length, Blurred, n_tracing_iterations, outlier_tolerance, 'linear',target_voxelsize ]

data = {nom: [valeur] for nom, valeur in zip(name_info_sup, info_sup)}

df1 = pd.DataFrame(traced_points[0])
df2 = pd.DataFrame(data)

writer = pd.ExcelWriter(excel_file)

# Écrire chaque DataFrame dans une feuille différente
df1.to_excel(writer, sheet_name='data', index=False)
df2.to_excel(writer, sheet_name='info', index=False)

# Fermer le writer
writer.save()

writer.close()


SyntaxError: EOL while scanning string literal (3818335879.py, line 3)